In [ ]:
spark

In [ ]:
# input params:
# opco[string]
# start_datetime[string] - ex: 1900-01-01 00:00:00
# end_datetime[string] - ex: 2026-02-09 06:45:00 ## In current implementation No end_datetime, they are filtering based on last run time, any data after that 
# run_datetime[string]
# [change timezone of emr]

In [1]:
from pyspark.sql import functions as f
from pyspark.sql.types import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1771847019502_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# define input params
opco="tx"
start_datetime=""
end_datetime=""
run_datetime="20260219_101235"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def display(df, n=10):
    return df.show(n=n, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2-source_interval_data_files.sh

In [4]:
#schema defintion for source data - tx
def get_nonvee_lg_tx_src_schema():
        #Base columns
        fields = [
            StructField("record_type", StringType(), True),
            StructField("record_version", StringType(), True),
            StructField("timestamp", StringType(), True),
            StructField("premise_id", StringType(), True),
            StructField("esiid", StringType(), True),
            StructField("provisioned", StringType(), True),
            StructField("meter_id", StringType(), True),
            StructField("purpose", StringType(), True),
            StructField("commodity", StringType(), True),
            StructField("units", StringType(), True),
            StructField("calculation_constant", StringType(), True),
            StructField("cinterval", StringType(), True),
            StructField("count", StringType(), True),
            StructField("firstintervaldatetime", StringType(), True)
        ]

        # Add the repeating t{n} and v{n} columns (1 to 100)
        for i in range(1,101):
            fields.append(StructField(f"t{i}", StringType(), True))
            fields.append(StructField(f"v{i}", StringType(), True))

        return StructType(fields)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_filtered_s3_paths():
    """
    Lists S3 prefixes and filters for those >= target_timestamp.
    
    """
    return \
    ["s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_1501/incr-itvl_20260209T130330.txt"
     ,"s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_1901/incr-itvl_20260209T170337.txt"
     ,"s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_2301/incr-itvl_20260209T210322.txt"
#      ,"s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_1101/incr-itvl_20260209T090323.txt"
#      ,"s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_0731/incr-itvl_20260209T050331.txt"
#      ,"s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_0301/incr-itvl_20260209T010354.txt"
    ]

#### stg_nonvee.interval_data_files_tx_src_vew.ddl conversion to pyspark


In [5]:

path = "s3://aep-datalake-raw-dev/util/intervals/nonvee/tx/20260209_1501/incr-itvl_20260209T130330.txt"

tx_source_df = spark.read \
            .format("csv") \
            .option("sep","~") \
            .option("quote", "\"") \
            .option("escape", "\\") \
            .option("nullValue", "") \
            .option("emptyValue", None) \
            .schema(get_nonvee_lg_tx_src_schema()) \
            .load(path)

# incr-itvl source data transformation

# Updated DST logic
# Parse source timestamp (ET local time, no timezone in data)
# Convert ET -> UTC -> CT using IANA zones so spark handles DST automatically

# interval_ts = f.to_timestamp("firstintervaldatetime", "MMddyyyyhhmmssa")

# interval_ct = f.from_utc_timestamp(
#                 f.to_utc_timestamp(interval_ts, "America/neew_York"), "America/Chicago"
# )

# Question: what is INPUT__FILE__NAME ?

value_cols = [f"v{i}" for i in range(1,101)]

tx_source_df = (
    tx_source_df
    .withColumn("filename", f.lit("INPUT__FILE__NAME"))
    .withColumn("value", f.split(f.concat_ws(",", *value_cols), ","))
    .withColumn("intervallength", f.col("cinterval").cast("int").cast("string"))
    .withColumn("aep_sec_per_intrvl", f.col("cinterval").cast("int"))
    .withColumn("interval_unix_datetime", f.unix_timestamp("firstintervaldatetime")) 
)
    
display(tx_source_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [ ]:
## 2 - source_interval_files.sh

In [ ]:
# ============================================================
# UOM Mapping — LEFT JOIN subquery
# ============================================================
uom_mapping_df = (
    spark.table("usage_nonvee.uom_mapping")
    .filter(f.col("aep_opco") == opco)
    .select(
        f.col("aep_opco"),
        f.col("aep_channel_id"),
        f.col("aep_derived_uom"),
        f.col("name_register"),
        f.col("aep_srvc_qlty_idntfr"),
        f.col("aep_raw_uom"),
        f.col("value_mltplr_flg")
    )
)

In [ ]:
# ============================================================
# meter_premise_macs_ami — LEFT JOIN subquery
# ============================================================
target_companies = ["94", "97"]

ami_df = (
    spark.table("stg_nonvee.meter_premise_macs_ami")
    .filter(f.col("co_cd_ownr").isin(target_companies))
    .select(
        f.col("prem_nb"),
        f.col("bill_cnst"),
        f.col("acct_clas_cd"),
        f.col("devc_cd"),
        f.col("pgm_id_nm"),
        f.concat(
            f.coalesce(f.col("tx_mand_data"), f.lit("")),
            f.coalesce(f.col("doe_nb"), f.lit("")),
            f.coalesce(f.col("serv_deliv_id"), f.lit("")),
        ).alias("sd"),
        f.col("mfr_devc_ser_nbr"),
        f.col("mtr_pnt_nb"),
        f.col("tarf_pnt_nb"),
        f.lit(None).cast(StringType()).alias("cmsg_mtr_mult_nb"),
        f.col("mtr_inst_ts"),
        f.col("mtr_rmvl_ts"),
        f.to_timestamp(f.col("mtr_inst_ts"), "yyyy-MM-dd HH:mm:ss[.SSSSSS]").cast("Long").alias("unix_mtr_inst_ts"),
        f.to_timestamp(f.col("mtr_rmvl_ts"), "yyyy-MM-dd").cast("Long").alias("unix_mtr_rmvl_ts"),
        f.to_timestamp(f.col("acct_turn_on_dt"), "yyyy-MM-dd").cast("Long").alias("unix_acct_turn_on_dt"),
        f.to_timestamp(f.col("acct_turn_off_dt"), "yyyy-MM-dd").cast("Long").alias("unix_acct_turn_off_dt"),
        f.col("serv_city_ad"),
        f.col("serv_zip_ad"),
        f.col("state_cd")
    )
)